In [ ]:
import gradio as gr
import requests
import io
from PIL import Image
import base64
import time
import random
import json

def generate_image(api_key, prompt, model_id="stabilityai/stable-diffusion-xl-base-1.0"):
    """
    Generate an image based on a text prompt using Hugging Face's API.
    
    Parameters:
    - api_key: Your Hugging Face API key
    - prompt: Description of the image to generate
    - model_id: The Hugging Face model ID to use
    """
    if not api_key.strip():
        return None, "Error: Please enter your Hugging Face API key"
    
    if not prompt.strip():
        return None, "Error: Please enter a prompt"
    
    try:
        # Add retry logic
        max_retries = 3
        retry_count = 0
        
        while retry_count < max_retries:
            try:
                # Set up the API call to Hugging Face
                API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
                headers = {
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json"
                }
                
                # Prepare the payload based on the model type
                if "controlnet" in model_id.lower():
                    payload = {
                        "inputs": {
                            "prompt": prompt,
                            "image": None  # You would need to add image handling for ControlNet
                        }
                    }
                elif "instruct-pix2pix" in model_id.lower():
                    payload = {
                        "inputs": {
                            "prompt": prompt,
                            "image": None  # You would need to add image handling for Pix2Pix
                        }
                    }
                else:
                    # Standard text-to-image model
                    payload = {
                        "inputs": prompt,
                        "parameters": {
                            "num_inference_steps": 30,
                            "guidance_scale": 7.5,
                            "width": 768,
                            "height": 768
                        }
                    }
                
                # Make the API request
                response = requests.post(API_URL, headers=headers, json=payload, timeout=90)
                
                # Handle different response types
                if response.status_code == 200:
                    # Success
                    if response.headers.get("content-type") == "application/json":
                        # Some models return JSON with image data
                        data = response.json()
                        if isinstance(data, list) and len(data) > 0 and "image" in data[0]:
                            img_data = base64.b64decode(data[0]["image"])
                        else:
                            return None, f"Error: Unexpected JSON response format from {model_id}"
                    else:
                        # Most models return the image directly
                        img_data = response.content
                    
                    # Create PIL image from the data
                    img = Image.open(io.BytesIO(img_data))
                    return img, f"Success! Image generated using {model_id}."
                    
                elif response.status_code == 503:
                    # Model is loading
                    retry_count += 1
                    wait_time = 2 ** retry_count
                    try:
                        data = response.json()
                        if "estimated_time" in data:
                            wait_time = max(wait_time, data["estimated_time"])
                    except:
                        pass
                    
                    time.sleep(wait_time)
                    continue
                    
                elif response.status_code == 429:
                    # Rate limit
                    retry_count += 1
                    time.sleep(2 ** retry_count)
                    continue
                    
                else:
                    return None, f"Error {response.status_code}: {response.text}"
            
            except requests.exceptions.RequestException as e:
                retry_count += 1
                if retry_count >= max_retries:
                    return None, f"Error: Connection failed after multiple attempts: {str(e)}"
                time.sleep(2 ** retry_count)
        
        # If we've reached here, we've exhausted our retries
        return None, "Error: Failed to generate image after multiple attempts"
            
    except Exception as e:
        return None, f"Error: {str(e)}"

def generate_prompt_from_image(api_key, input_image):
    """
    Generate a detailed prompt from an uploaded image using Hugging Face's image-to-text models.
    
    Parameters:
    - api_key: Hugging Face API key
    - input_image: The source image to analyze
    """
    if not api_key.strip():
        return "Error: Please enter your Hugging Face API key"
    
    if input_image is None:
        return "Error: Please upload an image"
    
    # Try with Hugging Face API
    try:
        # Convert to RGB if needed and resize if too large
        input_image = input_image.convert("RGB")
        max_size = 1024
        if input_image.width > max_size or input_image.height > max_size:
            input_image.thumbnail((max_size, max_size), Image.LANCZOS)
        
        # Convert image to bytes
        buffered = io.BytesIO()
        input_image.save(buffered, format="JPEG")
        image_bytes = buffered.getvalue()
        
        # Try different image captioning models
        models = [
            "Salesforce/blip-image-captioning-large",
            "nlpconnect/vit-gpt2-image-captioning",
            "microsoft/git-large-coco"
        ]
        
        for model_id in models:
            try:
                API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
                headers = {"Authorization": f"Bearer {api_key}"}
                
                response = requests.post(
                    API_URL,
                    headers=headers,
                    data=image_bytes,
                    timeout=30
                )
                
                if response.status_code == 200:
                    data = response.json()
                    
                    # Handle different response formats
                    if isinstance(data, list):
                        if len(data) > 0 and isinstance(data[0], dict) and "generated_text" in data[0]:
                            caption = data[0]["generated_text"]
                        elif len(data) > 0 and isinstance(data[0], str):
                            caption = data[0]
                        else:
                            continue
                    elif isinstance(data, dict) and "generated_text" in data:
                        caption = data["generated_text"]
                    else:
                        continue
                    
                    # Enhance the caption
                    return f"{caption}, highly detailed, professional, sharp focus, high resolution"
                
                elif response.status_code == 503:
                    # Model is loading, try the next one
                    continue
            except:
                continue
    except:
        pass
    
    # If all API attempts fail, use fallback method
    return generate_fallback_prompt(input_image)

def generate_fallback_prompt(image):
    """
    Generate a basic prompt from an image when API fails.
    Uses simple image analysis to create a description.
    """
    # Convert to small size for analysis
    img = image.copy()
    img.thumbnail((100, 100))
    
    # Simple color analysis
    colors = img.getcolors(10000)
    if colors:
        colors.sort(reverse=True, key=lambda x: x[0])
        dominant_colors = colors[:3]
        
        # Map RGB to basic color names
        color_names = []
        for count, (r, g, b) in dominant_colors:
            if r > 200 and g > 200 and b > 200:
                color_names.append("white")
            elif r < 60 and g < 60 and b < 60:
                color_names.append("black")
            elif r > 200 and g < 100 and b < 100:
                color_names.append("red")
            elif r < 100 and g > 200 and b < 100:
                color_names.append("green")
            elif r < 100 and g < 100 and b > 200:
                color_names.append("blue")
            elif r > 200 and g > 200 and b < 100:
                color_names.append("yellow")
            else:
                color_names.append("colorful")
        
        color_desc = " and ".join(list(set(color_names))[:2])
    else:
        color_desc = "colorful"
    
    # Brightness analysis
    brightness = sum(img.convert("L").getdata()) / (img.width * img.height)
    if brightness > 200:
        light_desc = "bright"
    elif brightness < 50:
        light_desc = "dark"
    else:
        light_desc = "balanced"
    
    # Aspect ratio for scene type guessing
    aspect = img.width / img.height
    if aspect > 1.2:
        scene_type = "landscape"
    elif aspect < 0.8:
        scene_type = "portrait"
    else:
        scene_type = "scene"
    
    # Random adjectives to enrich description
    adjectives = ["detailed", "beautiful", "stunning", "professional", "high-quality", 
                  "artistic", "creative", "impressive", "elegant", "dynamic"]
    
    # Generate prompt
    prompt = f"A {random.choice(adjectives)} {light_desc} {scene_type} with {color_desc} elements, highly detailed, professional photography, sharp focus, high resolution"
    
    return prompt

# Create Gradio interface
with gr.Blocks(title="Hugging Face Image Generator") as app:
    gr.Markdown("# Hugging Face Image Generator")
    
    with gr.Tabs():
        with gr.TabItem("Text to Image"):
            with gr.Row():
                with gr.Column():
                    api_key_txt = gr.Textbox(
                        label="API Key", 
                        placeholder="Enter your Hugging Face API key",
                        type="password"
                    )
                    prompt_txt = gr.Textbox(
                        label="Prompt", 
                        placeholder="Describe the image you want to generate",
                        lines=3
                    )
                    # Add model selection dropdown with popular Hugging Face models
                    model_txt = gr.Dropdown(
                        choices=[
                            "stabilityai/stable-diffusion-xl-base-1.0",
                            "runwayml/stable-diffusion-v1-5",
                            "CompVis/stable-diffusion-v1-4",
                            "stabilityai/stable-diffusion-2-1",
                            "dreamlike-art/dreamlike-diffusion-1.0",
                            "prompthero/openjourney",
                            "andite/anything-v4.0",
                            "hakurei/waifu-diffusion",
                            "dalle-mini/dalle-mini",
                            "sd-dreambooth-library/herge-style",
                            "nitrosocke/Ghibli-Diffusion",
                            "eimiss/EimisAnimeDiffusion_1.0v",
                            "sd-dreambooth-library/van-gogh-diffusion",
                            "facebook/bart-large-cnn",
                            "cjwbw/stable-diffusion-v1-5-lightning",
                            "stabilityai/sdxl-turbo",
                            "runwayml/stable-diffusion-inpainting",
                            "timbrooks/instruct-pix2pix"
                        ],
                        value="stabilityai/stable-diffusion-xl-base-1.0",
                        label="Select Model"
                    )
                    generate_btn = gr.Button("Generate Image", variant="primary")
                    
                with gr.Column():
                    output_image_txt = gr.Image(label="Generated Image")
                    output_message_txt = gr.Textbox(label="Status")
            
            # Add some examples
            gr.Examples(
                examples=[
                    ["A peaceful countryside with a small cottage"],
                    ["A young girl with a cat in a magical forest"],
                    ["A floating castle in the sky with airships"],
                    ["A quiet riverside town at sunset"],
                ],
                inputs=prompt_txt
            )
            
        with gr.TabItem("Prompt Generator"):
            with gr.Row():
                with gr.Column():
                    api_key_prompt = gr.Textbox(
                        label="API Key", 
                        placeholder="Enter your Hugging Face API key",
                        type="password"
                    )
                    input_image_prompt = gr.Image(
                        label="Upload Image to Generate Prompt", 
                        type="pil"
                    )
                    generate_prompt_btn = gr.Button("Generate Detailed Prompt", variant="primary")
                    generated_prompt = gr.Textbox(
                        label="Generated Prompt", 
                        lines=4,
                        placeholder="Your detailed prompt will appear here..."
                    )
                    use_prompt_btn = gr.Button("Use this Prompt to Generate Image")
                    # Add model selection dropdown for prompt tab
                    model_prompt = gr.Dropdown(
                        choices=[
                            "stabilityai/stable-diffusion-xl-base-1.0",
                            "runwayml/stable-diffusion-v1-5",
                            "CompVis/stable-diffusion-v1-4",
                            "stabilityai/stable-diffusion-2-1",
                            "dreamlike-art/dreamlike-diffusion-1.0",
                            "prompthero/openjourney",
                            "andite/anything-v4.0",
                            "hakurei/waifu-diffusion",
                            "dalle-mini/dalle-mini",
                            "sd-dreambooth-library/herge-style",
                            "nitrosocke/Ghibli-Diffusion",
                            "eimiss/EimisAnimeDiffusion_1.0v",
                            "sd-dreambooth-library/van-gogh-diffusion",
                            "facebook/bart-large-cnn",
                            "cjwbw/stable-diffusion-v1-5-lightning",
                            "stabilityai/sdxl-turbo",
                            "runwayml/stable-diffusion-inpainting",
                            "timbrooks/instruct-pix2pix"
                        ],
                        value="stabilityai/stable-diffusion-xl-base-1.0",
                        label="Select Model for New Image"
                    )
                
                with gr.Column():
                    output_image_prompt = gr.Image(label="Generated Image")
                    output_message_prompt = gr.Textbox(label="Status")
    
    # Add information about Hugging Face Models
    gr.Markdown("""
    ## Features:
    - **Text to Image**: Create images from your text descriptions using Hugging Face models
    - **Prompt Generator**: Upload an image to get a detailed prompt, then use that prompt to generate new images
    - **Multiple Models**: Choose from different Hugging Face diffusion models
    - The API key is only used to make requests to Hugging Face and is not stored
    
    ## Popular Models:
    - **stabilityai/stable-diffusion-xl-base-1.0**: High-quality SDXL model
    - **runwayml/stable-diffusion-v1-5**: Classic Stable Diffusion model
    - **nitrosocke/Ghibli-Diffusion**: Generates images in Studio Ghibli style
    - **prompthero/openjourney**: Midjourney-inspired style
    - **andite/anything-v4.0**: Anime-style generations
    - **hakurei/waifu-diffusion**: Anime character generations
    - **sd-dreambooth-library/herge-style**: Tintin comic style
    - **sd-dreambooth-library/van-gogh-diffusion**: Van Gogh painting style
    - **timbrooks/instruct-pix2pix**: Image editing model
    
    ## Note:
    - Some models may take longer to load if they haven't been used recently
    - If the Hugging Face server is experiencing issues, the app will use a fallback method to generate prompts
    - Different models may have different optimal prompt styles
    """)
    
    # Connect the button clicks to the functions
    generate_btn.click(
        fn=generate_image,
        inputs=[api_key_txt, prompt_txt, model_txt],
        outputs=[output_image_txt, output_message_txt]
    )
    
    generate_prompt_btn.click(
        fn=generate_prompt_from_image,
        inputs=[api_key_prompt, input_image_prompt],
        outputs=generated_prompt
    )
    
    use_prompt_btn.click(
        fn=generate_image,
        inputs=[api_key_prompt, generated_prompt, model_prompt],
        outputs=[output_image_prompt, output_message_prompt]
    )

# Launch the app
if __name__ == "__main__":
    app.launch()